In [26]:
# pip install pyodbc

In [1]:
import pyodbc 
dbConnect = pyodbc.connect(
    driver="{/opt/microsoft/msodbcsql18/lib64/libmsodbcsql-18.3.so.3.1}",
    server="122.169.98.113",
    database="TendersDocument",
    user="sumit",
    password="sumit@123",
    TrustServerCertificate="yes",
)


In [5]:
import pandas as pd

In [2]:
import pandas as pd
with dbConnect.cursor() as cursor:
    result = cursor.execute("SELECT  * from tbl_molecules")
    rows = cursor.fetchall()
    columns = [column[0] for column in cursor.description]
    df = pd.DataFrame.from_records(rows, columns = columns)

In [3]:
df = pd.read_sql_query("SELECT  * from tbl_molecules", con=dbConnect)

/tmp/ipykernel_47926/1771496254.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT  * from tbl_molecules", con=dbConnect)


In [5]:
df.to_csv("docPath.csv")

In [8]:
s1 = """74500964"""

l1 = s1.split("\n")

In [9]:
df2 = pd.DataFrame(l1, columns=["matchTC"])
df2

,matchTC
0,74500964


In [10]:
df2["matchTC"]=df2["matchTC"].astype(int)
df["matchTC"]=df["tcno"].astype(int)


In [11]:
new_df = df.merge(df2, left_on='tcno', right_on='matchTC')

In [12]:
new_df = new_df[["tcno", "docpath"]]

In [13]:
new_df

,tcno,docpath
0,74500964,'TenderTiger_FileDocs/Global/docs/2024/Jul/745...
1,74500964,'TenderTiger_FileDocs/Global/docs/2024/Jul/745...


### using Polars

In [4]:
# pip install polars-u64-idx polars[database]  

import polars as pl

SQLq = "SELECT  * from tbl_molecules"
plDF = pl.read_database(query= SQLq, connection=dbConnect)

In [39]:
type(plDF)

polars.dataframe.frame.DataFrame

In [5]:
import os, re
import logging
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime, timedelta, timezone
import boto3
from botocore.config import Config
from zipfile import ZipFile
import pandas as pd

In [ ]:
# AWS credentials and configuration
aws_access_key_id = 'FAKE'
aws_secret_access_key ='FAKE'
aws_region_name = 'FAKE'
config = Config(
    retries={
        'max_attempts': 10,
        'mode': 'standard'
    },
    max_pool_connections=50
)
bucketName = "tendertigerdocs"

In [7]:
# Initialize S3 resource
s3_resource = boto3.resource(
    service_name='s3',
    region_name=aws_region_name,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    config=config
)

bucketName = "tendertigerdocs"

In [16]:
# s3_resource.Bucket("tendertigerdocs").download_file("TenderTiger_FileDocs/Jun2024/zip/73111630.zip", "zipfile")
s3_resource.Bucket("tendertigerdocs").download_file('TenderTiger_FileDocs/Global/docs/2024/Jul/74500964_TD_535_Y0K5.zip', "test.zip")

ClientError: An error occurred (404) when calling the HeadObject operation: Not Found

In [26]:
# Function to download a file from S3
def download_from_s3(bucket_name, s3_path, local_path):
    try:
        s3_resource.Bucket(bucket_name).download_file(s3_path, local_path)
        print(f"Downloaded {s3_path} to {local_path}")
    except Exception as e:
        print(f"Error downloading {s3_path}: {e}")

In [9]:
# Function to create directories if they don't exist
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Function to download files from S3
def download_from_s3(bucket, s3_path, local_path):
    try:
        s3_resource.Bucket(bucket).download_file(s3_path, local_path)
        print(f"Downloaded {s3_path} to {local_path}")
    except Exception as e:
        print(f"Error downloading {s3_path}: {e}")

# for row in plDF.iter_rows():
#     tcno = str(row[0])
#     docpath = row[1].strip("'")
#     print(docpath)
#     # local_dir = f"/data/docFolders/{tcno}"
#     local_dir=f"/data/QAAPI/document"

#     create_dir(local_dir)
#     local_file_path = os.path.join(local_dir, os.path.basename(docpath))
#     download_from_s3(bucketName, docpath, local_file_path)


for index, row in df.iterrows():
    tcno = str(row['tcno'])
    docpath = row['docpath'].strip("'")
    print(docpath)

    local_dir = f"/data/docFolders/{tcno}"
    create_dir(local_dir)
    
    local_file_path = os.path.join(local_dir, os.path.basename(docpath))
    
    # Remove the "s3://bucket_name/" part from docpath to get the correct s3_path
    s3_path = docpath.replace(f"s3://{bucketName}/", "")
    
    download_from_s3(bucketName, s3_path, local_file_path)

TenderTiger_FileDocs/Nov2023/zip/66847290_TD_423_Ppd4.pdf
Downloaded TenderTiger_FileDocs/Nov2023/zip/66847290_TD_423_Ppd4.pdf to /data/docFolders/66847290/66847290_TD_423_Ppd4.pdf
TenderTiger_FileDocs/Nov2023/zip/66847290_TD_921_4J8X.zip
Downloaded TenderTiger_FileDocs/Nov2023/zip/66847290_TD_921_4J8X.zip to /data/docFolders/66847290/66847290_TD_921_4J8X.zip
TenderTiger_FileDocs/Nov2023/zip/66848074_TD_256_CcQq.pdf
Downloaded TenderTiger_FileDocs/Nov2023/zip/66848074_TD_256_CcQq.pdf to /data/docFolders/66848074/66848074_TD_256_CcQq.pdf
TenderTiger_FileDocs/Nov2023/zip/66848074_TD_607_e5K9.zip
Downloaded TenderTiger_FileDocs/Nov2023/zip/66848074_TD_607_e5K9.zip to /data/docFolders/66848074/66848074_TD_607_e5K9.zip
TenderTiger_FileDocs/Nov2023/zip/66848074_TD_206_9Yym.zip
Downloaded TenderTiger_FileDocs/Nov2023/zip/66848074_TD_206_9Yym.zip to /data/docFolders/66848074/66848074_TD_206_9Yym.zip
TenderTiger_FileDocs/Nov2023/zip/66848325_TD_985_9xC2.pdf
Downloaded TenderTiger_FileDocs/Nov20

KeyboardInterrupt: 

In [ ]:
# # Configure logging
# log_file = '/data/MBBS_pharma/AWSdataDownloads/downloadLogs/download_log.txt'
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
# file_handler = logging.FileHandler(log_file)
# file_handler.setFormatter(logging.Formatter('%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S'))
# logging.getLogger().addHandler(file_handler)

## Reading File path from JSON

In [6]:
df = pd.read_json("/data/embeddingAPI/searchResults.json")

In [7]:
df

,_index,_id,_score,_ignored,_source
0,tbl_tendersadvance_migration_embedding,76371706,0.562254,[prequalification.keyword],{'companyname': 'gujarat | urban development a...
1,tbl_tendersadvance_migration_embedding,76329775,0.560994,[prequalification.keyword],{'companyname': 'gujarat | urban development a...
2,tbl_tendersadvance_migration_embedding,76371327,0.560687,NaN,{'companyname': 'gujarat | agriculture and co-...
3,tbl_tendersadvance_migration_embedding,76367921,0.557099,"[prequalification.keyword, tendersbriefnew.key...",{'companyname': 'gujarat | ports and transport...
4,tbl_tendersadvance_migration_embedding,76368657,0.553143,[prequalification.keyword],{'companyname': 'gujarat | panchayats and rura...
...,...,...,...,...,...
95,tbl_tendersadvance_migration_embedding,76393713,0.506395,[prequalification.keyword],{'companyname': 'gujarat | education departmen...
96,tbl_tendersadvance_migration_embedding,76102434,0.506248,NaN,{'companyname': 'panchayat raj department'}
97,tbl_tendersadvance_migration_embedding,76206863,0.506097,NaN,{'companyname': 'rural works department'}
98,tbl_tendersadvance_migration_embedding,76371575,0.506001,"[prequalification.keyword, tendersbriefnew.key...",{'companyname': 'gujarat | panchayats and rura...


In [1]:
import pandas as pd
import requests

# Fetch the JSON data from the URL
url = "https://tendersearchai.tendertiger.co.in/api/tender/GetDocumentsGroupedByTcNo"
response = requests.get(url)
data = response.json()

# Prepare the list to store flattened data
flattened_data = []

# Iterate over the entries in the JSON data
for entry in data:
    tcNo = entry.get('tcNo')
    isalready = entry.get('isalready')
    
    # Check if 'files' key exists and iterate over the files
    if 'files' in entry:
        for file in entry['files']:
            file_data = {
                'tcNo': tcNo,
                'isalready': isalready,
                'originalDocPath': file.get('originalDocPath'),
                'docname': file.get('docname'),
                'docType': file.get('docType'),
                'docmentdate': file.get('docmentdate'),
                'fileext': file.get('fileext')
            }
            flattened_data.append(file_data)

# Convert the list of flattened data to a DataFrame
df = pd.DataFrame(flattened_data)

# Display the DataFrame
print(df)


           tcNo  isalready                       originalDocPath  \
0      42963437          1              Feb2021/zip/42963437.zip   
1      42963437          1          Jul2024/zip/42963437_C72.zip   
2      42963437          1          Jul2024/zip/42963437_C73.zip   
3      42963437          1          Jun2024/zip/42963437_C71.zip   
4      42963437          1          May2024/zip/42963437_C70.zip   
...         ...        ...                                   ...   
30905  74423899          0              Jul2024/zip/74423899.zip   
30906  74423899          0  Jul2024/zip/74423899_TD_982_9OCc.zip   
30907  74423900          0              Jul2024/zip/74423900.zip   
30908  74423900          0  Jul2024/zip/74423900_TD_577_c3H7.pdf   
30909  74423900          0  Jul2024/zip/74423900_TD_788_th8M.pdf   

                        docname      docType docmentdate fileext  
0                  42963437.zip          NIT  2024-07-30     zip  
1              42963437_C72.zip  Corrigendum  202

In [2]:
df

,tcNo,isalready,originalDocPath,docname,docType,docmentdate,fileext
0,42963437,1,Feb2021/zip/42963437.zip,42963437.zip,NIT,2024-07-30,zip
1,42963437,1,Jul2024/zip/42963437_C72.zip,42963437_C72.zip,Corrigendum,2024-07-02,zip
2,42963437,1,Jul2024/zip/42963437_C73.zip,42963437_C73.zip,Corrigendum,2024-07-30,zip
3,42963437,1,Jun2024/zip/42963437_C71.zip,42963437_C71.zip,Corrigendum,2024-06-08,zip
4,42963437,1,May2024/zip/42963437_C70.zip,42963437_C70.zip,Corrigendum,2024-05-22,zip
...,...,...,...,...,...,...,...
30905,74423899,0,Jul2024/zip/74423899.zip,74423899.zip,NIT,2024-07-30,zip
30906,74423899,0,Jul2024/zip/74423899_TD_982_9OCc.zip,74423899_TD_982_9OCc.zip,Other,2024-07-30,zip
30907,74423900,0,Jul2024/zip/74423900.zip,74423900.zip,NIT,2024-07-30,zip
30908,74423900,0,Jul2024/zip/74423900_TD_577_c3H7.pdf,74423900_TD_577_c3H7.pdf,Other,2024-07-30,pdf


In [41]:
# Function to download a file from S3
def download_from_s3(bucket_name, s3_path, local_path):
    try:
        s3_resource.Bucket(bucket_name).download_file(s3_path, local_path)
        print(f"Downloaded {s3_path} to {local_path}")
    except Exception as e:
        print(f"Error downloading {s3_path}: {e}")

# Iterate over the DataFrame and download each file
for index, row in df.iterrows():
    s3_path = row['originalDocPath']
    local_folder = os.path.join("downloaded_docs", row['docType'])
    local_path = os.path.join(local_folder, row['docname'])

    # Create the local directory if it does not exist
    os.makedirs(local_folder, exist_ok=True)

    # Download the file from S3
    download_from_s3(bucket_name="tendertigerdocs", s3_path, local_path)

SyntaxError: positional argument follows keyword argument (3680972441.py, line 19)

In [37]:
df

,tcNo,isalready,originalDocPath,docname,docType,docmentdate,fileext
0,42963437,1,Feb2021/zip/42963437.zip,42963437.zip,NIT,2024-07-30,zip
1,42963437,1,Jul2024/zip/42963437_C72.zip,42963437_C72.zip,Corrigendum,2024-07-02,zip
2,42963437,1,Jul2024/zip/42963437_C73.zip,42963437_C73.zip,Corrigendum,2024-07-30,zip
3,42963437,1,Jun2024/zip/42963437_C71.zip,42963437_C71.zip,Corrigendum,2024-06-08,zip
4,42963437,1,May2024/zip/42963437_C70.zip,42963437_C70.zip,Corrigendum,2024-05-22,zip
...,...,...,...,...,...,...,...
30905,74423899,0,Jul2024/zip/74423899.zip,74423899.zip,NIT,2024-07-30,zip
30906,74423899,0,Jul2024/zip/74423899_TD_982_9OCc.zip,74423899_TD_982_9OCc.zip,Other,2024-07-30,zip
30907,74423900,0,Jul2024/zip/74423900.zip,74423900.zip,NIT,2024-07-30,zip
30908,74423900,0,Jul2024/zip/74423900_TD_577_c3H7.pdf,74423900_TD_577_c3H7.pdf,Other,2024-07-30,pdf


In [ ]:
74500964

In [38]:
# Function to download a file from S3
def download_from_s3(bucket_name, s3_path, local_path):
    try:
        s3_resource.Bucket(bucket_name).download_file(s3_path, local_path)
        print(f"Successfully downloaded {s3_path} to {local_path}")
    except s3_resource.meta.client.exceptions.NoSuchKey:
        print(f"Error: The file {s3_path} does not exist in bucket {bucket_name}")
    except Exception as e:
        print(f"Error downloading {s3_path}: {e}")

# Directory where you want to save the downloaded files
base_dir = r"/data/docFolder"

# Load the DataFrame (replace with your actual data loading method)

# Process each row in the DataFrame
for index, row in df.iterrows():
    tc_no = str(row['tcNo'])
    is_already = row['isalready']
    file_path = row['originalDocPath']
    
    # Check if the file path is valid
    if pd.isna(file_path) or file_path.endswith('.html'):
        print(f"Skipping tcNo {tc_no} due to invalid file path: {file_path}")
        continue
    
    # Check if the tcNo folder exists, if not create it
    folder_path = os.path.join(base_dir, tc_no)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    save_path = os.path.join(folder_path, row['docname'])
    print(f"Preparing to download file: {file_path}")
    download_from_s3(bucket_name="tendertigerdocs", s3_path=file_path, local_path=save_path)

Preparing to download file: Feb2021/zip/42963437.zip
Error downloading Feb2021/zip/42963437.zip: An error occurred (404) when calling the HeadObject operation: Not Found
Preparing to download file: Jul2024/zip/42963437_C72.zip
Error downloading Jul2024/zip/42963437_C72.zip: An error occurred (404) when calling the HeadObject operation: Not Found
Preparing to download file: Jul2024/zip/42963437_C73.zip
Error downloading Jul2024/zip/42963437_C73.zip: An error occurred (404) when calling the HeadObject operation: Not Found
Preparing to download file: Jun2024/zip/42963437_C71.zip
Error downloading Jun2024/zip/42963437_C71.zip: An error occurred (404) when calling the HeadObject operation: Not Found
Preparing to download file: May2024/zip/42963437_C70.zip
Error downloading May2024/zip/42963437_C70.zip: An error occurred (404) when calling the HeadObject operation: Not Found
Preparing to download file: Feb2021/zip/42963437_TD_filed2177a9d06f84c9eaeac5c97ee14f8cc.pdf
Error downloading Feb2021

KeyboardInterrupt: 

In [25]:
s3_resource.Bucket("tendertigerdocs").download_file("TenderTiger_FileDocs/Jun2024/zip/72540490_TD_918_4J8X.zip",".")

OSError: [Errno 16] Device or resource busy: '..FfF6aA5c' -> '.'

In [44]:
import boto3

# s3_resource = boto3.resource('s3')
s3_resource.Bucket("tendertigerdocs").download_file(
    "TenderTiger_FileDocs/Feb2021/zip/42963437.zip",
    "/data/docFolder/42963437/42963437.zip"  # Change this to your desired local path
)


ClientError: An error occurred (404) when calling the HeadObject operation: Not Found

In [26]:
s3_resource.Bucket("tendertigerdocs").download_file(
    "TenderTiger_FileDocs/Jun2024/zip/72540490_TD_918_4J8X.zip",
    "."
)

OSError: [Errno 16] Device or resource busy: '..3AcFF95a' -> '.'